### Import PySpark
Nogle få kommentarer:
 - Download spark pre-built for hadoop 2.6, jeg vil også anbefale jer at bruge spark 1.6.0 da der er nogle problemer med 1.6.1 [hent den her http://www.apache.org/dyn/closer.lua/spark/spark-1.6.0/spark-1.6.0-bin-hadoop2.6.tgz]
 - husk at ændre paths i denne notebook
 - `os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages com.databricks:spark-avro_2.10:2.0.1 pyspark-shell"` vil give jer mulighed for direkte at loade avro filer

In [ ]:
import sys
import os
import os.path

SPARK_HOME = """spark-1.6.0-bin-hadoop2.6/""" ## PATH TO SPARK

sys.path.append(os.path.join(SPARK_HOME, "python", "lib", "py4j-0.9-src.zip"))
sys.path.append(os.path.join(SPARK_HOME, "python", "lib", "pyspark.zip"))
os.environ["SPARK_HOME"] = SPARK_HOME
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages com.databricks:spark-avro_2.10:2.0.1 pyspark-shell"
os.environ["PYSPARK_PYTHON"]="/usr/bin/python3"


from pyspark import SparkConf, SparkContext, StorageLevel
from pyspark.sql import SQLContext

conf = (SparkConf()
         .setMaster("local[*]")
         .setAppName("My app"))
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)

In [ ]:
# load data to dataframe
df = (sqlContext.read.format("com.databricks.spark.avro")
      .load("data/201512/*.avro")
      )

In [ ]:
# I kan se strukturen på data her
df.printSchema()

In [ ]:
# kig på data
df.show(10)

In [ ]:
# transform til rdd
data = df.rdd

In [ ]:
data.count()

In [ ]:
#more imports
from DatabaseHelper import DatabaseHelper
from dateutil import parser

db = DatabaseHelper()

first_period_min_date = parser.parse("2015-12-01 00:00:00+00:00")
first_period_max_date = parser.parse("2015-12-31 23:59:59+00:00")
second_period_min_date = parser.parse("2016-01-01 00:00:00+00:00")
second_period_max_date = parser.parse("2016-01-31 23:59:59+00:00")
third_period_min_date = parser.parse("2016-02-01 00:00:00+00:00")
third_period_max_date = parser.parse("2016-02-29 23:59:59+00:00")


In [ ]:
#filtering
swe_data = data.filter(lambda row: "Sweden" in row["country"]).filter(lambda row: parser.parse(row["start_time"]) >= first_period_min_date
                           and parser.parse(row["end_time"]) <= third_period_max_date)
# conversion
def convert_time_and_spatial(row):
    return [((db.calculate_spatial_bin(row["longitude"], row["latitude"]),x),[row["useruuid"]]) for x in db.calculate_time_bins(row["start_time"], row["end_time"])]

swe_data = swe_data.flatMap(convert_time_and_spatial)


In [ ]:
swe_data = swe_data.reduceByKey(lambda a, b: a+b if b[0] not in a else a) #key: (spatial_bin, time_bin), val: users
swe_data.take(100)